# Creating High Resolution Satellite Images using Mapbox 

source : https://towardsdatascience.com/creating-high-resolution-satellite-images-with-mapbox-and-python-750b3ac83dd7

Done by Toby Zaw 

In [1]:
import mercantile
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import requests 
import shutil
from tqdm.notebook import tqdm
%matplotlib inline

## Import your latitude and longitude

In [2]:
#Converting form Lat/Lng to tilesets

lat_lng = [16.8409, 96.1735]

In [49]:
delta=0.001
tl = [lat_lng[0]+delta, lat_lng[1]-delta]
br = [lat_lng[0]-delta, lat_lng[1]+delta]
z = 17# Set the resolution (max at 15)

In [50]:
print('top left :',tl)
print('bottom right',br)


top left : [16.841900000000003, 96.1725]
bottom right [16.8399, 96.17450000000001]


In [51]:
import mercantile
tl_tiles = mercantile.tile(tl[1],tl[0],z)
br_tiles = mercantile.tile(br[1],br[0],z)

x_tile_range = [tl_tiles.x,br_tiles.x];print(x_tile_range)
y_tile_range = [tl_tiles.y,br_tiles.y];print(y_tile_range)

[100551, 100552]
[59313, 59314]


In [53]:
import requests # The requests package allows use to call URLS
import shutil   # shutil will be used to copy the image to the local
# Loop over the tile ranges
for i,x in tqdm(enumerate(range(x_tile_range[0],x_tile_range[1]+1))):
      for j,y in tqdm(enumerate(range(y_tile_range[0],y_tile_range[1]+1))):
            # Call the URL to get the image backr = requests.get('https://api.mapbox.com/v4/mapbox.terrain-
            r = requests.get('https://api.mapbox.com/v4/mapbox.terrain-rgb/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.pngraw?access_token=pk.eyJ1IjoidG9ieXphd3RodWh0ZXQiLCJhIjoiY2t1MnF6bjdoMHNsbTJwcWhpYWZjZWxpciJ9.lMU2n-qA9stK2C0v32rBFQ', stream=True)
            # Next we will write the raw content to an image
            with open('C:/Users/Toby/Desktop/New folder/Mapbox/elevation_images/' + str(i) + '.' + str(j) + '.png','wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f) 
                # Do the same for the satellite data
                r =requests.get('https://api.mapbox.com/v4/mapbox.satellite/'+str(z)+'/'+str(x)+'/'+str(y)+'@2x.pngraw?access_token=pk.eyJ1IjoidG9ieXphd3RodWh0ZXQiLCJhIjoiY2t1MnF6bjdoMHNsbTJwcWhpYWZjZWxpciJ9.lMU2n-qA9stK2C0v32rBFQ', stream=True)
            with open('C:/Users/Toby/Desktop/New folder/Mapbox/satellite_images/' + str(i) + '.' + str(j) + '.png','wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [48]:
# Import the image, math and os libraries
import PIL
import math
from os import listdir
from os.path import isfile, join
# Loop over the elevation and satellite image set
for img_name in ['elevation','satellite']:
    # Make a list of the image names   
    image_files = ['C:/Users/Toby/Desktop/New folder/Mapbox/'+img_name+'_images/' + f for f in listdir('C:/Users/Toby/Desktop/New folder/Mapbox/'+img_name+'_images/')]
    # Open the image set using pillow
    images = [PIL.Image.open(x) for x in image_files]
    # Calculate the number of image tiles in each direction
    edge_length_x = x_tile_range[1] - x_tile_range[0]
    #print('x edge length',edge_length_x)
    edge_length_y = y_tile_range[1] - y_tile_range[0]
    #print('y edge length',edge_length_y)
    edge_length_x = max(1,edge_length_x)
    edge_length_y = max(1,edge_length_y)
    # Find the final composed image dimensions  
    width, height = images[0].size
    total_width = width*edge_length_x
    total_height = height*edge_length_y
    #print('total width',total_width)
    #print('total height', total_height)
    # Create a new blank image we will fill in
    composite = PIL.Image.new('RGB', (total_width, total_height))
    # Loop over the x and y ranges
    y_offset = 0
    for i in range(0,edge_length_x):
        x_offset = 0
    for j in range(0,edge_length_y):
        # Open up the image file and paste it into the composed image at the given offset position
        tmp_img = PIL.Image.open('C:/Users/Toby/Desktop/New folder/Mapbox/'+img_name+'_images/' + str(i) +'.' + str(j) + '.png')
        composite.paste(tmp_img, (y_offset,x_offset))
        x_offset += width # Update the width
        y_offset += height # Update the height
# Save the final image
composite.save('C:/Users/Toby/Desktop/New folder/Mapbox/composite_images/'+img_name+'.png')

x edge length 1
y edge length 0
total width 512
total height 512
x edge length 1
y edge length 0
total width 512
total height 512
